# Introduction
Converting a university Assignment in Jupyter Notebook on how to use CNNs into a Google Colab notebook.

The aim is to train a image classification model on the [Inaturalist 12K Dataset](https://storage.googleapis.com/wandb_datasets/nature_12K.zip).

The dataset features:  

*   10 classes
*   10k images

The dataset will be split into 90% training and 10% for validation.

The original assignment utilised wandb for experimental tracking and reporting on the model during it's training and testing phase. However, I have removed the code as I do not have access to a wandb API key.

Original Repo:
https://github.com/PranjalChitale/CS6910_Assignment2/tree/main



# Setup
Importing the required libraries.

In [ ]:
import os
import numpy as np
import matplotlib as plt
import tensorflow as tf
import pandas as pd
from tensorflow.keras import layers,models
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import different pretrained image-classification models.
https://keras.io/api/applications/

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.resnet import ResNet50
from keras.applications.xception import Xception
# from keras.applications.vgg16 import VGG16
from keras.applications.mobilenet_v2 import MobileNetV2


Verifying the tensorflow version is latest 2.15.0


In [ ]:
print(tf.__version__)



2.15.0


# Downloading the Dataset


Downloading the inaturalist dataset.

In [ ]:
dataset_url = "https://storage.googleapis.com/wandb_datasets/nature_12K.zip"
dataset_dir = tf.keras.utils.get_file("nature_12K",origin=dataset_url,cache_dir='.',extract=True)


3816687935/3816687935 [==============================] - 42s 0us/step


# Preprocessing


### Setting Directories
Setting the train dataset and test dataset directory.

In [ ]:
trainset_dir = './datasets/inaturalist_12K/train/'
testset_dir = './datasets/inaturalist_12K/val/'
classlist = [name for name in os.listdir(trainset_dir) if os.path.isdir(os.path.join(trainset_dir, name))]

### Generating the train dataset and val dataset


In [ ]:
def generate_batch_train_val(path, augmentation, batch_size, image_size):
    rescaledata = tf.keras.layers.Rescaling(1./127.5, offset=-1)
    #Splits the dataset into train and validation.
    #Keras' ImageDataGenerator is used to split data into train and test.
    if augmentation:
        #Applies data augmentation if specified
        train_data_gen = ImageDataGenerator(
                            rescale = 1./255,
                            horizontal_flip = True,
                            rotation_range = 30,
                            shear_range = 0.2,
                            zoom_range = 0.2,
                            width_shift_range = 0.2,
                            height_shift_range = 0.2,
                            validation_split = 0.1,
                        )
    else:
        train_data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.10)

    """
    Flow from directory expects that images belonging to each class is present in its own folder
    but inside the same parent folder : data directory. It takes path to the data directory as
    input and generates batches of desired batch size. Need to specify appropriate subset
    (training / validation) to generate batches for respective subset.
    """

    train_data = train_data_gen.flow_from_directory(
            path,
            target_size=image_size,
            color_mode="rgb",
            batch_size=batch_size,
            class_mode="sparse",
            shuffle=True,
            seed = 0,
            subset="training"
        )

    val_data = train_data_gen.flow_from_directory(
        path,
        target_size=image_size,
        color_mode="rgb",
        batch_size=batch_size,
        class_mode="sparse",
        shuffle=True,
        seed=0,
        subset="validation"
    )

    #Gets the list of class labels.
    class_labels = list(train_data.class_indices.keys())


    return train_data, val_data, class_labels

### Generating the test dataset

In [ ]:
def generate_batch_test(path, batch_size, image_size):
    #Generates batches of test data.
    test_data_gen = ImageDataGenerator(
    rescale = 1./255
    )

    test_data = test_data_gen.flow_from_directory(
            path,
            target_size=image_size,
            color_mode="rgb",
            batch_size=batch_size,
            class_mode="sparse",
            shuffle=True,
            seed=0,
        )

    return test_data

# Training
This includes training predicting and logging the experiments.



In [11]:
def train(config = None ):

    # Reading and setting the configuation
    batch_size = config['batch_size']
    augmentation = config['augmentation']
    pretrain_model = config['pretrain_model']
    droprate = config['droprate']
    batch_norm = config['batch_normalization']
    epoch = config['epoch']
    fc_size = config["fc_size"]
    num_of_trainable_layers = config['num_of_trainable_layers']

    # Choosing the pretrained model based on configuration input.
    if pretrain_model == 'InceptionV3':
        image_size = (299,299)
        base_model = tf.keras.applications.InceptionV3(include_top = False,weights='imagenet', input_shape=image_size+(3,))

    elif pretrain_model == 'InceptionResNetV2':
        image_size = (299,299)
        base_model = tf.keras.applications.InceptionResNetV2(include_top = False,weights='imagenet',input_shape=image_size+(3,))

    elif pretrain_model == 'ResNet50':
        image_size = (224,224)
        base_model = tf.keras.applications.ResNet50(include_top = False,weights='imagenet',input_shape=image_size+(3,))

    elif pretrain_model == 'Xception':
        image_size = (299,299)
        base_model = tf.keras.applications.Xception(include_top = False,weights='imagenet',input_shape=image_size+(3,))

    elif pretrain_model == 'MobileNetV2':
        image_size = (224,224)
        base_model = tf.keras.applications.MobileNetV2(include_top = False,weights='imagenet',input_shape=image_size+(3,))

    # Freezing the pretrained model's layer.
    base_model.trainable = False

    # Adding the new fully connected layer on top of the feature extraction layers of pretrained model.
    model = tf.keras.Sequential([
        tf.keras.Input(shape=image_size+(3,)),
        base_model,
        Flatten(),
        Dense(fc_size,activation='relu'),
    ])

    if batch_norm:
        model.add(BatchNormalization())

    model.add(Dropout(droprate))
    model.add(Dense(fc_size, activation='relu'))
    model.add(Dropout(droprate))
    train_data,val_data,class_labels = generate_batch_train_val(trainset_dir, augmentation, batch_size,image_size)
    model.add(Dense(len(class_labels) ,activation='softmax'))

    # Setting the optimisation and loss function.
    model.compile(
        optimizer= 'adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )

    # Dividing the epoch between pretraining and fine-tuning(if asked).
    if num_of_trainable_layers > 0:
        fine_tuning_epoch = int(epoch/2)
        pretrain_epoch = int(epoch/2)
    else:
        pretrain_epoch = epoch

    # Training the model.
    hist=model.fit(train_data,epochs=pretrain_epoch,validation_data=val_data)#,callbacks=[wandb_callback]


    # Fine-tuning
    # Based on input, if number of trainable layers are >0, then setting that number of the freezed layers in pretrained model trainable.
    if num_of_trainable_layers > 0:
        num_of_trainable_layers=num_of_trainable_layers+(len(model.layers)-len(base_model.layers))
        for layer in reversed(model.layers):
            if(num_of_trainable_layers> 0):
                layer.trainable=True
                num_of_trainable_layers -= 1

        model.compile(
            optimizer= tf.keras.optimizers.Adam(learning_rate=1e-5),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(),
            metrics=['accuracy'])

        # Fine tuning.
        hist=model.fit(train_data,epochs=fine_tuning_epoch,validation_data=val_data)



# Running
### Best parameters configuration run.

In [12]:
config = {
    'pretrain_model': 'Xception',
    'epoch':9,
    'batch_size': 16,
    'augmentation': True,
    'fc_size': 256,
    'droprate':0.4,
    'batch_normalization': True,
    'num_of_trainable_layers' : 1
    }
train(config)

83683744/83683744 [==============================] - 0s 0us/step
Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/4
154/563 [=======>......................] - ETA: 47:46 - loss: 1.6310 - accuracy: 0.4841

KeyboardInterrupt: 

### Sweep configuration

In [ ]:
sweep_config = {
    'name': 'A2_B_bayes',
    'method': 'bayes',
    'early_terminate':{'type': 'hyperband', 'min_iter': 3},
    'metric':{'name':'val_Accuracy','goal':'maximize'},
    'parameters': {
        'pretrain_model' : {'values' :['InceptionV3','InceptionResNetV2','ResNet50','Xception','MobileNetV2']},
        'epoch' : {'values':[6,9]},
        'batch_size' : {'values':[16,32,128]},
        'augmentation':{'values':[True,False]},
        'fc_size': {'values': [128,256,512]},
        'droprate':{'values': [0.4,0.5]},
        'batch_normalization': {'values':[True,False]},
        'num_of_trainable_layers' : {'values': [0,1,2]},

    }
}